In [ ]:
import json
import re
import tomllib
from pathlib import Path
from typing import Any

import httpx

# from rich.pretty import pprint

### Helper Functions
#### Write API Data Function

In [ ]:
def write_api_data(
    endpoint_name: str, response: httpx.Response, data_dir: Path
) -> None:
    """Write an API response's JSON to file.

    Parameters
    ----------
    endpoint_name : str
        API endpoint name.
    response : httpx.Response
        Response returned from the GET request.
    data_dir : Path
        Path to the data directory for storing the output JSON file.
    """
    # location of the output JSON file.
    file_path = data_dir.joinpath(f"{endpoint_name}.json")
    with open(file_path, "w") as f:
        json.dump(response.json(), f)

# Extract
### Paths

In [ ]:
backend_dir = Path.cwd().parent
data_dir = backend_dir / "data"

### Read Secrets File Containing API Access Token and Account ID

In [ ]:
with open(backend_dir / "secrets.toml", "rb") as f:
    secrets = tomllib.load(f)

### Base URL and Headers

In [ ]:
# Base URL for the tradier API.
base_url = "https://api.tradier.com/v1"
# Headers with access token.
headers = {
    "Authorization": f"Bearer {secrets['tradier']['access_token']}",
    "Accept": "application/json",
}

## Account Endpoints
### Build API Request Data

In [ ]:
# Account information fragment.
account_fragment = f"/accounts/{secrets['tradier']['account_id']}"
# Fragments of account endpoints.
endpoint_fragments = {
    "user_profile": "/user/profile",
    "balances": f"{account_fragment}/balances",
    "positions": f"{account_fragment}/positions",
    "history": f"{account_fragment}/history",
    "gain_loss": f"{account_fragment}/gainloss",
    "orders": f"{account_fragment}/orders",
}

# Request data with url and headers for each request.
request_data: dict[str, dict[str, Any]] = {}
for endpoint_name, endpoint in endpoint_fragments.items():
    request_data[endpoint_name] = {
        "url": f"{base_url}{endpoint}",
        "headers": headers,
    }
# Add any parameters if necessary.
request_data["history"]["params"] = {"limit": 1000}
request_data["gain_loss"]["params"] = {"limit": 1000}

### Request Data from API

In [ ]:
for endpoint_name, request in request_data.items():
    response = httpx.get(**request)
    if endpoint_name == "history":
        history = response.json()
    write_api_data(endpoint_name, response, data_dir)

### Collect Symbol Descriptions
#### Set of Stock Symbols

In [ ]:
symbol_pattern = r"^[A-Z]+(?![a-z])"
symbols = set()
for transaction in history["history"]["event"]:
    if transaction.get("trade"):
        match = re.match(symbol_pattern, transaction["trade"]["symbol"])
        symbols.add(match.group(0))
    elif transaction.get("option"):
        match = re.match(symbol_pattern, transaction["option"]["description"])
        symbols.add(match.group(0))

#### Build API Request Data

In [ ]:
# Fragment of lookup symbol endpoint.
endpoint = "/markets/lookup"

# Request data with url and headers for each request.
request_data: dict[str, dict[str, Any]] = {}
for symbol in symbols:
    request_data[symbol] = {
        "url": f"{base_url}{endpoint}",
        "headers": headers,
        "params": {"q": symbol},
    }

#### Request Data from API

In [ ]:
symbol_descriptions = {}
for symbol, request in request_data.items():
    response = httpx.get(**request)
    security = response.json()["securities"]["security"]
    if isinstance(security, list):
        for element in security:
            if element["symbol"] == symbol:
                symbol_descriptions[symbol] = element["description"]
            break
    else:
        symbol_descriptions[symbol] = security["description"]

#### Write Descriptions to File

In [ ]:
# location of the output JSON file.
file_path = data_dir.joinpath("symbol_descriptions.json")
with open(file_path, "w") as f:
    json.dump(symbol_descriptions, f)

## Market Endpoints
### Build API Request Data

In [ ]:
# Fragments of account endpoints.
endpoint_fragments = {
    "historical_quotes": "/markets/history",
}

# Request data with url and headers for each request.
request_data = {}
for endpoint_name, endpoint in endpoint_fragments.items():
    request_data[endpoint_name] = {
        "url": f"{base_url}{endpoint}",
        "headers": headers,
    }
# Add any parameters if necessary.
request_data["historical_quotes"]["params"] = {"symbol": "AAPL"}

#### Request Data from API

In [ ]:
for endpoint_name, request in request_data.items():
    response = httpx.get(**request)
    write_api_data(endpoint_name, response, data_dir)